In [2]:
import os
import pandas as pd
import numpy as np
import re
import string
import json
# -*- coding:utf-8 -*-

# 合并不同物种

In [61]:
doen = pd.read_excel("output/文献合并/human_label.xlsx",index_col=0)

In [65]:
add_list = [i for i in os.listdir("output/文献标注-校对") if "_add" in i] # add为校对好的文本
doen = pd.read_excel("output/human_label_1.0.xlsx",index_col=0)
doen_y = doen.loc[doen["是否是致病菌"]=="yes"]
for i in add_list:

    a =  pd.read_excel("output/文献标注-校对/%s" % i)
    print(a.shape, a.drop_duplicates().shape)
    b = a.loc[a["是否是致病菌"]=="是"]

    print(i,doen_y.loc[doen_y["物种名称"].isin(a["物种名称"]),"ID"].shape[0],doen.loc[doen["物种名称"].isin(a["物种名称"]),"ID"].shape[0],4)


(176, 11) (176, 11)
Curved_add.xls 21 168 4
(549, 11) (549, 11)
GramPr_add.xls 210 506 4
(889, 11) (889, 11)
GramN_add.xls 215 802 4
(413, 11) (413, 11)
Anaerobic_add.xls 56 384 4
(296, 11) (296, 11)
GramPc_add.xls 22 257 4
(96, 11) (96, 11)
Intracellular_add.xls 12 94 4


In [53]:
add_list = [i for i in os.listdir("output/文献标注-校对") if "_add" in i] # add为校对好的文本
doen = pd.read_excel("output/human_label_1.0.xlsx",index_col=0)
doen_y = doen.loc[doen["是否是致病菌"]=="yes"]
spec_l = []

for i in add_list:

    a =  pd.read_excel("output/文献标注-校对/%s" % i)
    b = a.loc[a["是否是致病菌"]=="是"]
    spec_l.extend(list(a["物种名称"]))
    print(i,len(set(doen_y.loc[doen_y["物种名称"].isin(a["物种名称"]),"物种名称"])),len(set(doen.loc[doen["物种名称"].isin(a["物种名称"]),"物种名称"])),4)
    # print(set(a["物种名称"]))



Curved_add.xls 12 67 4
GramPr_add.xls 106 185 4
GramN_add.xls 103 264 4
Anaerobic_add.xls 34 154 4
GramPc_add.xls 17 88 4
Intracellular_add.xls 8 31 4


In [59]:
c = [i for i in spec_l if i not in set(doen["物种名称"])]

In [3]:
add_list = [i for i in os.listdir("output/文献标注-校对") if "_add" in i] # add为校对好的文本
col_dic =  {"样本量":"num","患者性别":"gender","年龄":"age","免疫状态":"immu","证据等级":"evi"}

for col_name, col_ab in col_dic.items():
    
    ori_l = [i for i in os.listdir("output/文献标注/") if col_ab in i]
    df_l = []
    
    for add in add_list:
        
        # 读取校对后的文档
        sped = add.replace("_add.xls","")
        df = pd.read_excel("output/文献标注-校对/%s" % add, header=0, index_col=0)
        df["ID"] = df.index
        # 读取含有支持证据的文档
        file_names = [i for i in ori_l if sped in i]
        dfs = pd.read_excel("output/文献标注/%s" %file_names[0], header=0, index_col=0)
        
        # 将支持证据添加至校对后的文档
        df[['%s文本' % col_name, 'old_start', 'old_end', '所在句子', 'new_start','new_end']] = dfs[['answer', 'old_start', 'old_end', 'sup_evi', 'new_start','new_end']]
        
        # 为添加原文
        for idx in df.index:
            with open("../2.Label_prepare/output/paper_ld/%s/%s.txt" % (sped, idx), "r") as f:
                txt = f.read()
            df.loc[idx, "Full"] = txt 
        df_l.append(df)

    dfm = pd.concat(df_l,axis=0)
    dfm.index = [str(dfm.index[i]) + ":" + dfm["物种名称"].iloc[i] for i in range(dfm.shape[0])]
    dfm.to_excel("output/文献合并/human_label_%s.xlsx" % col_ab)

# 合并不同标注

In [13]:
col_dic =  {"样本量":"num","患者性别":"gender","年龄":"age","免疫状态":"immu","证据等级":"evi"}
dfs_l = []
for col_name, col_ab in col_dic.items():
    dfm = pd.read_excel("output/文献合并/human_label_%s.xlsx" % col_ab, index_col=0)
    if col_ab == "evi":
        dfs_l.append(dfm[['%s文本' % col_name,"检测方式文本"]])
    else:
        dfm[["%s_start"%col_ab, "%s_end"%col_ab]] = dfm[["old_start", "old_end"]]
        dfs_l.append(dfm[['%s文本' % col_name, "%s_start"%col_ab, "%s_end"%col_ab]])
dfs = pd.concat(dfs_l,axis=1)

dfh = dfm[['物种名称', '是否是致病菌', '样本量', '患者性别', '年龄', '免疫状态','预后', '证据等级','检测方式', 'id','ID', "Full"]]
dfh = pd.concat([dfh,dfs],axis=1)
dfh = dfh.fillna("none")

In [14]:
dfh = dfh.loc[~(dfh['是否是致病菌']== "无法确定"),:]

dfh.loc[dfh['是否是致病菌']== "是", "是否是致病菌"] = "yes"
dfh.loc[dfh['是否是致病菌'] != "yes", "是否是致病菌"] = "no"

dfh = dfh.copy()
dfh['本文中是肺部病原的概率'] = 0
dfh.loc[dfh["是否是致病菌"]=="yes",'本文中是肺部病原的概率'] = 1
dfh['标注方式'] = '人工标注'
dfh['原文链接'] = dfh['id']

In [15]:
# 数据清洗
dfh.loc[dfh["样本量"]==1, "样本量"] =  "1-9"
dfh.loc[dfh["样本量"]=="1-10", "样本量"] =  "1-9"
dfh.loc[dfh["样本量"]=="2-10", "样本量"] =  "1-9"
dfh.loc[dfh["样本量"]=="10-100", "样本量"] =  "10-99"
dfh.loc[dfh["样本量"]=="10-101", "样本量"] =  "10-99"
dfh.loc[dfh["样本量"]=="10-102", "样本量"] =  "10-99"
dfh.loc[dfh["样本量"]=="11-100", "样本量"] =  "10-99"
dfh.loc[dfh["样本量"]=="＞100", "样本量"] =  "100以上"
dfh.loc[dfh["样本量"]==">100", "样本量"] =  "100以上"


dfh.loc[dfh["年龄"]=="1-17", "年龄"] =  "1-17岁"
dfh.loc[dfh["年龄"]=="17-65", "年龄"] =  "18-65岁"
dfh.loc[dfh["年龄"]=="65以上", "年龄"] =  ">65岁"

dfh.loc[dfh["患者性别"]=="群体", "患者性别"] =  "none"

In [16]:
dfh.loc[dfh["免疫状态"]=="损伤", "免疫状态"] =  "抑制"
dfh.loc[dfh["免疫状态"]=="受损", "免疫状态"] =  "抑制"
dfh.loc[dfh["免疫状态"]=="正常", "免疫状态"] =  "none"

dfh["full"] = [i.lower() for i in dfh["Full"].tolist()]
dfh.loc[(dfh["Full"].str.contains("fibrosis")) & (dfh["是否是致病菌"]=="yes"),"免疫状态"] = "抑制"

In [17]:
dfh.loc[dfh["预后"]=="无法确定", "预后"] = "none"
dfh.loc[dfh["预后"]=="治愈", "预后"] =  "none"

dfh.loc[dfh["证据等级"]=="8：文献报道", "证据等级"] =  "none"
dfh.loc[dfh["证据等级"]=="7：免疫学证据", "证据等级"] =  "3：血及无菌体液（如脓胸液）"
dfh.loc[dfh["证据等级"]=="5-2：气管吸出物或痰的分子诊断（PCR、基因组测序等）病原", "证据等级"] =  "3：血及无菌体液（如脓胸液）"
dfh.loc[dfh["证据等级"]=="4：气管镜活检组织培养出病原体,或者支气管肺泡灌洗液（BALF）定量培养>10**4", "证据等级"] =  "4：气管镜活检组织培养出病原体，或者支气管肺泡灌洗液（BALF）定量培养>10**4"

In [18]:
# 保存
dfh[['物种名称', 'ID', '本文中是肺部病原的概率', '是否是致病菌', '样本量', '样本量文本','num_start','num_end','年龄', '年龄文本', 'age_start','age_end','患者性别', '患者性别文本', 'gender_start','gender_end','免疫状态', '预后','证据等级', '证据等级文本','检测方式', '检测方式文本', 'Full', '原文链接', '标注方式']].to_excel("output/文献合并/human_label_pat.xlsx")

文献合并中的human_label未扩展的1869篇；
用于Bert训练的为扩展后，修改了错误的物种名称后，去除重复后的合并2169篇